In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import os
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from hyperopt import hp, fmin, tpe, Trials
import numpy as np

In [2]:
os.chdir('C:\\Users\\jeetg\\code\\horse race prediction')

In [3]:
horse_df = pd.read_csv("horses.csv")
race_df = pd.read_csv('races.csv')

In [4]:
forward_df=pd.read_csv('cleaned_forward.csv')

In [5]:
pd.set_option('display.max_columns', 500)

In [6]:
horse_df.head()

,rid,horseName,age,saddle,decimalPrice,isFav,trainerName,jockeyName,position,positionL,dist,weightSt,weightLb,overWeight,outHandicap,headGear,RPR,TR,OR,father,mother,gfather,runners,margin,weight,res_win,res_place,price
0,271018,Combermere,6.0,0.0,0.222222,0,R G Frost,J Frost,1,NaN,NaN,11,0,NaN,NaN,NaN,111.0,94.0,NaN,Absalom,Queen's Parade,Sovereign Path,14,1.521003,69,1.0,1.0,NaN
1,271018,Royal Battery,6.0,0.0,0.090909,0,D H Barons,S Earle,2,10,NaN,11,0,NaN,NaN,NaN,101.0,88.0,NaN,Norfolk Air,All At Sea,Man The Rail,14,1.521003,69,0.0,1.0,NaN
2,271018,Just So,7.0,0.0,0.029412,0,J D Roberts,S Burrough,3,15,25.0,10,7,NaN,NaN,NaN,86.0,71.0,NaN,Sousa,Just Camilla,Ascertain I,14,1.521003,66,0.0,1.0,NaN
3,271018,Mandraki Shuffle,8.0,0.0,0.090909,0,Oliver Sherwood,M Richards,4,20,45.0,11,0,NaN,NaN,b,66.0,65.0,NaN,Mandalus,Indictment,Desert Call,14,1.521003,69,0.0,0.0,NaN
4,271018,Turnberry Dawn,8.0,0.0,0.047619,0,T B Hallett,P Richards,5,dist,75.0,11,0,NaN,NaN,NaN,NaN,45.0,NaN,Fair Turn,Shuil Alainn,Levanter,14,1.521003,69,0.0,0.0,NaN


In [7]:
race_df.head()

,rid,course,time,date,title,rclass,band,ages,distance,condition,hurdles,prizes,winningTime,prize,metric,countryCode,ncond,class,currency
0,271018,Exeter,03:15,90/01/01,David Garrett Memorial Challenge Trophy Novice...,NaN,NaN,6-8yo,3m1f,Soft,19 fences,"[2922.5, 875.0, 420.0, 192.5]",398.3,4409.0,5028.0,GB,5,0,NaN
1,275156,Tramore (IRE),02:00,90/01/01,Tattersalls Mares E.B.F. Novice Chase,NaN,NaN,NaN,2m,Soft,12 fences,[],267.0,NaN,3218.0,IE,5,0,NaN
2,282203,Catterick,02:45,90/01/01,Scotch Corner Handicap Chase,NaN,0-125,NaN,1m7½f,Good To Firm,12 fences,"[2238.0, 618.0, 294.0]",238.0,3150.0,3116.5,GB,2,0,NaN
3,298761,Cheltenham,02:30,90/01/01,A. S. W. Handicap Hurdle,NaN,NaN,NaN,2m,Good To Firm,NaN,"[5117.0, 1526.0, 728.0, 329.0]",243.8,7700.0,3218.0,GB,2,0,NaN
4,301118,Windsor,03:30,90/01/01,Touchen End Handicap Hurdle,NaN,0-115,NaN,2m6f,Good,NaN,"[2344.8, 652.8, 314.4]",330.7,3312.0,4424.0,GB,1,0,NaN


In [8]:
forward_df.head()

,course,countryCode,marketTime,title,runners,condition,prize,rclass,horseName,trainerName,jockeyName,RPRc,TRc,OR,weightSt,weightLb,age,decimalPrice
0,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,Class 5,All Down To Rosie,Conor O'Dwyer,Kevin Manning,0.367568,0.402174,0.422857,9,2,2,50.000000
1,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,Class 5,Colfer Kay,K J Condon,W J Lee,0.416216,0.461957,0.422857,9,2,2,6.037778
2,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,Class 5,Dha Leath,Garvan Donnelly,J M Sheridan,0.367568,0.402174,0.422857,9,2,2,49.666667
3,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,Class 5,Ellabella,Andrew McNamara,Colin Keane,0.367568,0.402174,0.422857,9,2,2,17.944444
4,Limerick,0,2020-09-11 12:45:00+01:00,Irish Stallion Farms EBF Fillies Maiden (Plus ...,14,Yielding To Soft,13717.5,Class 5,Fermoy,Mrs John Harrington,Tom Madden,0.383784,0.396739,0.422857,9,2,2,17.594737


In [9]:
horse_df.isnull().sum()

rid                   0
horseName             0
age                 257
saddle            84727
decimalPrice          0
isFav                 0
trainerName        1705
jockeyName           77
position              0
positionL        286392
dist             280622
weightSt              0
weightLb              0
overWeight       974075
outHandicap      968608
headGear         892820
RPR              317190
TR               388996
OR               554974
father             1032
mother             1654
gfather            5731
runners               0
margin                0
weight                0
res_win               0
res_place             0
price           1012202
dtype: int64

In [10]:
race_df.isnull().sum()

rid                 0
course              0
time                0
date                0
title               1
rclass         152992
band           231309
ages            13054
distance            0
condition          48
hurdles        274953
prizes              0
winningTime         0
prize           19708
metric              0
countryCode         0
ncond               0
class               0
currency       387291
dtype: int64

In [11]:
forward_df.isnull().sum()

course          0
countryCode     0
marketTime      0
title           0
runners         0
condition       0
prize           0
rclass          0
horseName       0
trainerName     0
jockeyName      0
RPRc            0
TRc             0
OR              0
weightSt        0
weightLb        0
age             0
decimalPrice    0
dtype: int64

In [106]:
infodf = pd.merge(horse_df, race_df, on='rid', how='left')
infodf = pd.merge(infodf, forward_df, on=['horseName'], how='left')

In [76]:
merged_df = pd.merge(horse_df, race_df, on='rid', how='left')

In [77]:
merged_df = pd.merge(merged_df, forward_df, on=['horseName'], how='left')

In [78]:
merged_df.shape

(1020287, 63)

In [79]:
merged_df.columns

Index(['rid', 'horseName', 'age_x', 'saddle', 'decimalPrice_x', 'isFav',
       'trainerName_x', 'jockeyName_x', 'position', 'positionL', 'dist',
       'weightSt_x', 'weightLb_x', 'overWeight', 'outHandicap', 'headGear',
       'RPR', 'TR', 'OR_x', 'father', 'mother', 'gfather', 'runners_x',
       'margin', 'weight', 'res_win', 'res_place', 'price', 'course_x', 'time',
       'date', 'title_x', 'rclass_x', 'band', 'ages', 'distance',
       'condition_x', 'hurdles', 'prizes', 'winningTime', 'prize_x', 'metric',
       'countryCode_x', 'ncond', 'class', 'currency', 'course_y',
       'countryCode_y', 'marketTime', 'title_y', 'runners_y', 'condition_y',
       'prize_y', 'rclass_y', 'trainerName_y', 'jockeyName_y', 'RPRc', 'TRc',
       'OR_y', 'weightSt_y', 'weightLb_y', 'age_y', 'decimalPrice_y'],
      dtype='object')

In [80]:
# Drop columns with too many missing values
merged_df = merged_df.drop(['rid', 'decimalPrice_x', 'isFav', 'positionL', 'dist', 'weightSt_x', 'weightLb_x', 'overWeight', 'outHandicap', 'headGear', 'father', 'mother', 'gfather', 'runners_x','margin', 'weight','price', 'time',
                            'date',  'rclass_x', 'band', 'ages', 'distance','prizes', 'winningTime', 'prize_x', 'metric','countryCode_x', 'ncond', 'class', 'currency', 'course_y',
                            'countryCode_y', 'marketTime', 'title_y', 'runners_y', 'condition_y','saddle','TRc', 'RPRc', 'OR_y','hurdles',
                            'prize_y', 'rclass_y', 'trainerName_y', 'jockeyName_y', 'weightSt_y', 'weightLb_y', 'age_y', 'decimalPrice_y'], axis=1)

In [81]:
merged_df.columns

Index(['horseName', 'age_x', 'trainerName_x', 'jockeyName_x', 'position',
       'RPR', 'TR', 'OR_x', 'res_win', 'res_place', 'course_x', 'title_x',
       'condition_x'],
      dtype='object')

In [82]:
merged_df.isnull().sum()

horseName             0
age_x               257
trainerName_x      1730
jockeyName_x         77
position              0
RPR              319616
TR               391995
OR_x             559514
res_win               0
res_place             0
course_x              0
title_x               7
condition_x         219
dtype: int64

In [83]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')
merged_df[['TR', 'RPR', 'OR_x']] = imputer.fit_transform(merged_df[['TR', 'RPR', 'OR_x']])

In [84]:
# Impute missing values for numerical columns
merged_df['age_x'].fillna(merged_df['age_x'].median(), inplace=True)

In [85]:
# Impute missing values for categorical columns
merged_df['condition_x'].fillna(merged_df['condition_x'].mode()[0], inplace=True)

In [86]:
merged_df['trainerName_x'].fillna('Unknown', inplace=True)
merged_df['jockeyName_x'].fillna('Unknown', inplace=True)
merged_df['title_x'].fillna('Unknown', inplace=True)

In [87]:
print(merged_df['position'].value_counts())

position
1     98147
2     97866
3     97041
4     93683
40    87225
5     86565
6     77780
7     68618
8     59361
9     50543
10    42432
11    35233
12    28909
13    23324
14    18544
15    14555
16    11195
17     8287
18     6283
19     4413
20     3195
21     2009
22     1495
23     1045
24      761
25      548
26      416
27      304
28      213
29      138
30       69
31       31
32       19
33        9
34        7
35        6
0         6
36        4
37        3
38        3
39        2
Name: count, dtype: int64


In [88]:
# Encode categorical variables
le = LabelEncoder()
cat_cols = ['horseName', 'jockeyName_x', 'trainerName_x', 'course_x', 'condition_x','title_x']

for col in cat_cols:
    merged_df[col] = le.fit_transform(merged_df[col])

In [89]:
df = merged_df

In [91]:
df['avg_rating']=(df['OR_x']+df['RPR']+df['TR'])/3

In [93]:
from sklearn.model_selection import train_test_split

X = df.drop(['res_win','res_place','position','OR_x','TR','RPR'], axis=1)  # features                                                                                                                                                                                                                                                                                                                                                 
y = df['res_win']  # target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [95]:
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

In [96]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Separate numerical and categorical columns
X_train_scaled_num = pd.DataFrame(scaler.fit_transform(X_train[num_cols]), columns=num_cols)
X_test_scaled_num = pd.DataFrame(scaler.transform(X_test[num_cols]), columns=num_cols)

X_train_scaled = pd.concat([X_train_scaled_num, X_train[cat_cols]], axis=1)
X_test_scaled = pd.concat([X_test_scaled_num, X_test[cat_cols]], axis=1)

In [97]:
X_train_scaled.columns

Index(['age_x', 'avg_rating', 'horseName', 'jockeyName_x', 'trainerName_x',
       'course_x', 'condition_x', 'title_x'],
      dtype='object')

In [69]:
X_train_scaled = X_train_scaled.loc[:, ~X_train_scaled.columns.duplicated()]

In [98]:
scaler = StandardScaler()
scaler.fit(X_train_scaled)

StandardScaler()

In [99]:
min_samples = min(len(X_train_scaled), len(y_train))
X_train_scaled = X_train_scaled[:min_samples]
y_train = y_train[:min_samples]

In [100]:
column_order = X_train_scaled.columns.tolist()

# Reorder the columns in X_test
X_test_reordered = X_test[column_order]

In [103]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
clf.fit(X_train_scaled, y_train)

# Calculate the accuracy of the model
y_pred = clf.predict(X_test_reordered)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy:.3f}')

Model accuracy: 0.725


In [125]:
def predict_outcome(age_x,horseName, jockeyName_x, trainerName_x, course_x, condition_x,title_x,avg_rating):
    # Create a dictionary to store the input data
    input_data = {
        'age': [age_x],
        'horse_name': [horseName],
        'jockey_name': [jockeyName_x],
        'course': [course_x],
        'course_condition': [condition_x],
        'trainer_name': [trainerName_x],
        'title': [title_x],
        'average_rating': [round(float(avg_rating))]
    }

    # Convert the input data into a Pandas DataFrame
    input_df = pd.DataFrame(input_data)

    # Label encode the categorical columns (horse_name, jockey_name, course, course_condition)
    le = LabelEncoder()
    input_df['horse_name'] = le.fit_transform(input_df['horse_name'])
    input_df['jockey_name'] = le.fit_transform(input_df['jockey_name'])
    input_df['trainer_name'] = le.fit_transform(input_df['trainer_name'])
    input_df['course'] = le.fit_transform(input_df['course'])
    input_df['course_condition'] = le.fit_transform(input_df['course_condition'])
    input_df['title'] = le.fit_transform(input_df['title'])
    # Scale the input data using the same scaler used during training
    # Assume you have a scaler stored in a variable called `scaler`
    input_scaled = scaler.transform(input_df[['age','horse_name', 'jockey_name','trainer_name', 'course', 'course_condition','title','average_rating']].values)

    # Make a prediction using the trained model
    prediction = clf.predict(input_scaled)
    return prediction

In [127]:
# Example usage
age= ['10','4','6']
horse_name = ['Culleendubh', 'Baltray', 'Strong Suspicion']
jockey_name = ['P L Malone', 'G Kilfeather', 'K Morgan']
trainer_name = ['Richard Hannon Snr','M C Pipe','J Berry'] 
course = 'Fairyhouse (IRE)'
course_condition = 'Good'
title = 'Grunwick Stakes National Hunt Flat'
avg_rating = ['100','66','70']
for age, horse_name, jockey_name, trainer_name, avg_rating in zip(age, horse_name, jockey_name, trainer_name, avg_rating):
    outcome = predict_outcome(age, horse_name, jockey_name, trainer_name, course, course_condition, title, avg_rating)
    print(f"Predicted outcome for {horse_name} and {jockey_name}: {outcome}")

Predicted outcome for Culleendubh and P L Malone: [0.]
Predicted outcome for Baltray and G Kilfeather: [0.]
Predicted outcome for Strong Suspicion and K Morgan: [0.]
